In [1]:
%load_ext dotenv
%dotenv

In [2]:
import lyricsgenius
genius = lyricsgenius.Genius()
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()
import os
import pickle

In [5]:
SQL_FILEPATH = os.getenv("PLAYBACK_FILE")
lyrics_db = sqlite3.connect(SQL_FILEPATH)

In [6]:
# Load Data

In [7]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM tracks", lyrics_db)
print(df.head())

   index  year              artist                                 track  \
0      0  1944           Red Foley                    Smoke On The Water   
1      1  1944  The King Cole Trio           Straighten Up And Fly Right   
2      2  1944        Louis Jordan  Is You Is or Is You Ain't (Ma' Baby)   
3      3  1944          Tex Ritter           I'm Wastin' My Tears on You   
4      4  1944        Louis Jordan                          Ration Blues   

   rank                                               link  
0     1  /charts/country/video/1944/red-foley-smoke-on-...  
1     2  /charts/country/video/1944/the-king-cole-trio-...  
2     3  /charts/country/video/1944/louis-jordan-is-you...  
3     4  /charts/country/video/1944/tex-ritter-im-wasti...  
4     5  /charts/country/video/1944/louis-jordan-ration...  


## Create Unique Songs Dataframe

In [8]:
df.rename(columns = {'index':'track_id'}, inplace=True)

In [9]:
print(df.head())

   track_id  year              artist                                 track  \
0         0  1944           Red Foley                    Smoke On The Water   
1         1  1944  The King Cole Trio           Straighten Up And Fly Right   
2         2  1944        Louis Jordan  Is You Is or Is You Ain't (Ma' Baby)   
3         3  1944          Tex Ritter           I'm Wastin' My Tears on You   
4         4  1944        Louis Jordan                          Ration Blues   

   rank                                               link  
0     1  /charts/country/video/1944/red-foley-smoke-on-...  
1     2  /charts/country/video/1944/the-king-cole-trio-...  
2     3  /charts/country/video/1944/louis-jordan-is-you...  
3     4  /charts/country/video/1944/tex-ritter-im-wasti...  
4     5  /charts/country/video/1944/louis-jordan-ration...  


## Find Lyrics

In [10]:
def find_lyrics(row):
    try:
        return genius.search_song(row["track"], row["artist"])
    except:
        return "Error"

In [11]:
df["lyrics"] = df.progress_apply(find_lyrics, axis=1)
df.to_pickle("22-04-21-lyrics-pickle")

  0%|          | 0/7418 [00:00<?, ?it/s]

In [12]:
df.to_sql("tracks2", lyrics_db)

InterfaceError: Error binding parameter 7 - probably unsupported type.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7418 entries, 0 to 7417
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   track_id       7418 non-null   int64 
 1   year           7418 non-null   int64 
 2   artist         7418 non-null   object
 3   track          7418 non-null   object
 4   rank           7418 non-null   int64 
 5   link           7418 non-null   object
 6   lyrics_object  7121 non-null   object
 7   lyrics         7094 non-null   object
dtypes: int64(3), object(5)
memory usage: 463.8+ KB


In [15]:
df = df.rename({"lyrics": "lyrics_object"}, axis=1)
df["lyrics"] = df.progress_apply(
    lambda row: row["lyrics_object"].lyrics if row["lyrics_object"] and not isinstance(row["lyrics_object"],
                                                                                       str) else None, axis=1)

  0%|          | 0/7418 [00:00<?, ?it/s]

In [17]:
df.drop("lyrics_object", axis=1).to_sql("lyrics", lyrics_db)

7418